In [3]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException

driver=webdriver.Edge()

url='https://zu.fang.com/house-a015277-b02315/'
driver.get(url)

In [4]:
string_list=[]

i=0

while i<9:
    table =driver.find_element(By.CSS_SELECTOR, '#listBox > div.houseList')
    # Extract table houses
    houses = table.find_elements(By.TAG_NAME, 'dl')

    # Initialize lists to store data
    data = []

    # Loop through rows and extract data
    for house in houses:
        title = house.find_element(By.CSS_SELECTOR, "p.title").text
        price = house.find_element(By.CSS_SELECTOR, "span.price").text
        details = house.find_element(By.CSS_SELECTOR, "p.font15.mt12.bold").text
        data.append({"title": title, "price": price, "details": details})

    df = pd.DataFrame(data)
    string_list.append(df)
    #driver.refresh()
    try:
        next_page=driver.find_element(By.CSS_SELECTOR,'#rentid_D10_01 > a:nth-child(7)')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break

In [5]:
driver.quit()

In [6]:
df_full=pd.concat(string_list)
df_full

,title,price,details
0,清河小营桥电力宿舍南向精装次卧室出租,1800,合租次卧|3户合租|15㎡|朝南
1,整租 · 清河 · 永泰园新地标 · 4室2厅,16000,整租|4室2厅|239㎡|朝南
2,相寓 清河·世华龙樾一期西区·高楼层·2居室,7500,整租|2室1厅|78㎡|朝南北
3,整租 · 清河 · 永泰东里 · 2室1厅,6100,整租|2室1厅|57㎡|朝南北
4,相寓 清河·海清园·高楼层·3居室·次卧1,2800,合租单间|3户合租|15㎡|朝南
...,...,...,...
54,合租:主卧海淀清河永泰园新地标129平方米 3室2厅,2300,合租主卧|2户合租|129㎡|朝东南
55,海淀清河宝盛里91.9平方米 3室2厅1卫,7800,整租|3室2厅|91㎡|朝南北
56,合租:次卧海淀清河北砂小区89平方米 3室3厅2卫,2400,合租次卧|2户合租|89㎡|朝南
57,海淀清河高巢46平方米 1室1厅1卫,4800,整租|1室1厅|46㎡|朝南


In [9]:
df_full['area'] = df_full['details'].str.extract(r'(\d+\.?\d*)\s*㎡').astype(float)

In [10]:
df_full

,title,price,details,area
0,清河小营桥电力宿舍南向精装次卧室出租,1800.0,合租次卧|3户合租|15㎡|朝南,15.0
1,整租 · 清河 · 永泰园新地标 · 4室2厅,16000.0,整租|4室2厅|239㎡|朝南,239.0
2,相寓 清河·世华龙樾一期西区·高楼层·2居室,7500.0,整租|2室1厅|78㎡|朝南北,78.0
3,整租 · 清河 · 永泰东里 · 2室1厅,6100.0,整租|2室1厅|57㎡|朝南北,57.0
4,相寓 清河·海清园·高楼层·3居室·次卧1,2800.0,合租单间|3户合租|15㎡|朝南,15.0
...,...,...,...,...
54,合租:主卧海淀清河永泰园新地标129平方米 3室2厅,2300.0,合租主卧|2户合租|129㎡|朝东南,129.0
55,海淀清河宝盛里91.9平方米 3室2厅1卫,7800.0,整租|3室2厅|91㎡|朝南北,91.0
56,合租:次卧海淀清河北砂小区89平方米 3室3厅2卫,2400.0,合租次卧|2户合租|89㎡|朝南,89.0
57,海淀清河高巢46平方米 1室1厅1卫,4800.0,整租|1室1厅|46㎡|朝南,46.0


In [11]:
df_full[['price', 'area']].to_csv(
    'rent_prices_area.csv',  # 文件名
    index=False,              # 不保存索引
    encoding='utf-8-sig'      # 支持中文的编码格式
)

saved_df = pd.read_csv('rent_prices_area.csv')